You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [12]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [13]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [14]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [15]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [16]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [17]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [18]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [19]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [20]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [21]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2024-10-24 13:19:47,090 - 139765751077760 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [22]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Importance of staying updated on the latest trends and key players in the AI space

Key Points:
1. Latest Trends in Artificial Intelligence
- Advancements in machine learning algorithms
- Integration of AI in healthcare, finance, transportation, and other sectors
- Rise of ethical AI and responsible AI practices

2. Key Players in the A

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) is revolutionizing various industries with its ability to automate tasks, analyze data, and make decisions without human intervention. As AI continues to evolve, staying informed about the latest trends and key players in the industry is crucial for professionals, business leaders, and students interested in the field. The impact of AI on society is undeniable, and keeping up with the advancements in this field is essential for anyone looking to navigate the ever-changing landscape of technology.

## Latest Trends in Artificial Intelligence
Advancements in machine learning algorithms are driving the development of AI applications across sectors such as healthcare, finance, transportation, and more. The integration of AI into these industries is streamlining processes, improving efficiency, and enhancing decision-making. Moreover, 

- Display the results of your execution as markdown in the notebook.

In [23]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) is revolutionizing various industries with its ability to automate tasks, analyze data, and make decisions without human intervention. As AI continues to evolve, staying informed about the latest trends and key players in the industry is crucial for professionals, business leaders, and students interested in the field. The impact of AI on society is undeniable, and keeping up with the advancements in this field is essential for anyone looking to navigate the ever-changing landscape of technology.

## Latest Trends in Artificial Intelligence
Advancements in machine learning algorithms are driving the development of AI applications across sectors such as healthcare, finance, transportation, and more. The integration of AI into these industries is streamlining processes, improving efficiency, and enhancing decision-making. Moreover, the rise of ethical AI and responsible AI practices is gaining momentum, ensuring that AI technologies are developed and deployed ethically and responsibly. It is important for individuals to be aware of these trends and developments to understand the potential of AI in shaping the future of various industries.

## Key Players in the AI Industry
Leading companies like Google, Amazon, Microsoft, and IBM are at the forefront of AI research and development, investing heavily in innovative AI solutions. Additionally, startups specializing in AI applications are making significant contributions to the industry by introducing novel technologies and services. Academic institutions are also playing a crucial role in driving AI innovation through research and collaboration with industry partners. The collaboration between these key players is essential for pushing the boundaries of AI technologies and creating impactful solutions for real-world challenges.

## Noteworthy News in Artificial Intelligence
Recent breakthroughs in natural language processing and computer vision are expanding the capabilities of AI systems, enabling them to understand and interpret human language and visual information more effectively. Regulatory developments are shaping the landscape of AI adoption, with guidelines and policies being implemented to ensure the ethical and responsible use of AI technologies. Success stories of AI implementations in real-world scenarios are showcasing the tangible benefits of AI in improving processes, enhancing decision-making, and driving innovation. It is important to highlight these achievements to demonstrate the positive impact of AI on various industries and society as a whole.

In conclusion, staying informed about the latest trends, key players, and noteworthy news in the AI industry is essential for professionals, business leaders, and students looking to navigate the evolving landscape of AI. By keeping abreast of developments in AI, individuals can better understand the impact of AI on society, explore new opportunities in the field, and contribute to the responsible advancement of AI technologies. The continuous evolution of AI presents exciting possibilities for innovation and growth, and it is crucial to stay informed and engaged in this dynamic field.

**Call to Action:** Explore further resources on AI to deepen your understanding of the industry and stay informed about the latest trends and innovations. Let's embrace the future of AI together and shape a better tomorrow through responsible and ethical advancements in technology.

By following this comprehensive content plan, the blog article on Artificial Intelligence will engage the target audience with informative and relevant insights on the latest trends, key players, and noteworthy news in the AI industry. Let's continue to explore the endless possibilities of AI and its potential to transform industries and society for the better.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [24]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Ultimate Guide to YOUR TOPIC HERE

Introduction:
- Brief explanation of the importance and relevance of YOUR TOPIC HERE
- Mention of the latest trends and key players in the industry

Key Points:
1. Overview of YOUR TOPIC HERE
- Definition and significance
- Evolution of YOUR TOPIC HERE over time
- Current challenges and opportunities

2. Latest Trends in YOUR TOPIC HERE
- Analysis of the most recent developments in the industry
- Implications for businesses and consumers
- Predictions for future trends



I now can give a great answer.

Final Answer:

# The Ultimate Guide to Artificial Intelligence

## Introduction

In today's fast-paced world, understanding Artificial Intelligence (AI) is more crucial than ever. As technology continues to advance and shape our daily lives, keeping up with the latest trends and key players in the industry is essential. Whether you're a professional looking to stay ahead of the curve, a student eager to learn about the field, or a curious consumer seeking reliable information, this ultimate guide will provide you with valuable insights and knowledge.

## Overview of Artificial Intelligence

Artificial Intelligence is defined as the simulation of human intelligence processes by machines, especially computer systems. Its significance lies in its ability to perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. Over time, AI has evolved from simple rule-based systems

In [25]:
Markdown(result)

# The Ultimate Guide to Artificial Intelligence

## Introduction

In today's fast-paced world, understanding Artificial Intelligence (AI) is more crucial than ever. As technology continues to advance and shape our daily lives, keeping up with the latest trends and key players in the industry is essential. Whether you're a professional looking to stay ahead of the curve, a student eager to learn about the field, or a curious consumer seeking reliable information, this ultimate guide will provide you with valuable insights and knowledge.

## Overview of Artificial Intelligence

Artificial Intelligence is defined as the simulation of human intelligence processes by machines, especially computer systems. Its significance lies in its ability to perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. Over time, AI has evolved from simple rule-based systems to complex neural networks and deep learning algorithms. However, with evolution comes challenges and opportunities. Currently, the industry faces ethical concerns regarding AI's impact on privacy and job displacement, but also presents opportunities for growth and innovation in various sectors such as healthcare, finance, and transportation.

## Latest Trends in Artificial Intelligence

The latest trends in Artificial Intelligence have been nothing short of groundbreaking. With advancements in natural language processing, computer vision, and reinforcement learning, businesses and consumers alike are experiencing AI-powered products and services that were once considered science fiction. Looking towards the future, it is predicted that AI will continue to revolutionize industries by enabling predictive analytics, personalized recommendations, and autonomous systems that can enhance efficiency and decision-making processes.

## Key Players in Artificial Intelligence

Leading the pack in Artificial Intelligence are companies and individuals who have made significant contributions to the field. Their impact on the industry can be seen through groundbreaking research, innovative products, and strategic partnerships. Companies like Google, Microsoft, and Tesla, along with researchers such as Yoshua Bengio and Fei-Fei Li, have shaped the landscape of AI and continue to push boundaries in machine learning and robotics. By understanding the key players in Artificial Intelligence, one can gain valuable insights into the industry as a whole and explore potential opportunities for collaboration and competition.

## Noteworthy News in Artificial Intelligence

Recent news articles and studies related to Artificial Intelligence have shed light on the potential applications of AI in healthcare, the ethical implications of autonomous vehicles, and the rise of AI assistants in daily life. The implications of these findings for stakeholders are vast, ranging from improved patient care and safety to concerns about data privacy and algorithmic bias. Further reading and research are recommended for those looking to delve deeper into the complex and evolving field of Artificial Intelligence.

## Audience Analysis

Professionals, students, and general consumers interested in Artificial Intelligence all share a common goal: to stay informed and access reliable resources. By addressing their interests and pain points, this guide aims to provide valuable information and insights for all audiences. Whether you're a data scientist looking to expand your knowledge, a student exploring career opportunities in AI, or a curious individual interested in the future of technology, this guide offers a comprehensive overview of Artificial Intelligence.

## Call to Action

Engage with the content by sharing your thoughts and questions in the comments section below. Explore related articles, resources, and events for further learning. Subscribe to the blog for future updates on Artificial Intelligence. By following this comprehensive guide, readers will gain a deeper understanding of AI and stay informed on the latest trends and developments in the industry.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).